In [10]:
import chainer
from chainer import training, iterators, optimizers, serializers, Chain
import chainer.functions as F
import chainer.links as L
 
from chainer.training import extensions
from chainer.datasets import tuple_dataset
from chainer import optimizers, Chain, dataset, datasets, iterators
import numpy as np

In [14]:
def data_read( file_name, key):
    teachers = np.array([] )
    answers =  np.array([] )

    f = open( file_name, mode = "r" )
    f_string = f.readlines()
    data = np.array([] )
    
    for i in range( 0, len( f_string ) ):
        #引数を用いて正解ラベルを振り分ける
        tmp_data = np.zeros(10)
        if( i != 0 and i%key == 0 ):
            teachers = np.append( teachers, data)
            data = np.array([] )
            for j in f_string[i].replace( "\n", "" ):
                tmp_data[int(j)] += 1
            answers = np.append( answers, tmp_data )
        else:
            for j in f_string[i].replace( "\n", "" ):
                tmp_data[int(j)] += 1
                data = np.append( data, tmp_data )
    
    f.close()
    
    teachers = teachers.astype( np.float32 )
    answers = answers.astype( np.float32 )
    
    teachers = np.reshape( teachers, ( int( len( teachers ) / 10 / key ), key, 10 ) )
    
    answers = np.reshape( answers, ( int( len( answers ) / 10 ) , 10 ) )
    return teachers, answers


In [90]:
teachers, answers = data_read( 'numbers.txt', 5)

In [91]:
print(len(teachers))
print(len(answers))
print(teachers[0])
print(answers[0])

2667
1111
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]
[0. 0. 1. 0. 0. 0. 0. 1. 0. 1.]


In [92]:
#ニューラルネットワークの構築。
class RNN(Chain):
    
    def __init__(self, n_hidden, n_output):
        super(RNN, self).__init__(
            l1=L.LSTM(None, n_hidden),
            l2=L.Linear(None, n_output),
        )
        
    def reset_state(self):
        self.l1.reset_state()
        
    def __call__(self, x, t):
        y = self.predict(x)
        loss = F.mean_squared_error(y, t)
        #accuracy = F.accuracy(y, t)
        #chainer.reporter.report({'accuracy':accuracy},self)
        chainer.reporter.report({'loss':loss},self)
        return loss
    
    def predict(self, x):
        if train:
            h1 = F.dropout(self.l1(x),ratio = 0.5)
        else:
            h1 = self.l1(x)
        return self.l2(h1)

In [93]:
#Updaterを拡張する
from chainer import Variable, reporter

class LSTMUpdater(training.StandardUpdater):
    def __init__(self, data_iter, optimizer, device=None):
        super(LSTMUpdater,self).__init__(data_iter, optimizer, device=None)
        self.device = device
        
    def update_core(self):
        data_iter = self.get_iterator("main")
        optimizer = self.get_optimizer("main")
        
        batch = data_iter.__next__()
        x_batch, t_batch = chainer.dataset.concat_examples(batch, self.device)
        
        optimizer.target.reset_state()           
        optimizer.target.cleargrads()
        #loss = optimizer.target(Variable(x_batch), Variable(t_batch))
        loss = optimizer.target(x_batch, t_batch)
        loss.backward()
        loss.unchain_backward()                  
        optimizer.update() 


In [94]:
#　教師データのtupleを作成する
data = list(zip(teachers, answers))
N = len(data)
n_batchsize = 30
n_epoch = 100

#モデルを使う準備。オブジェクトを生成
n_hidden = 10
n_output = 10
model = RNN(n_hidden, n_output)
optimizer = optimizers.Adam()
optimizer.setup(model)

#学習用データと検証用データに分ける
train, test = chainer.datasets.split_dataset_random(data, int(N * 0.8))
train_iter = chainer.iterators.SerialIterator(train, n_batchsize, shuffle=False)
test_iter = chainer.iterators.SerialIterator(test, n_batchsize, repeat=False, shuffle=False)
updater = LSTMUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (n_epoch, "epoch"), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"])) # エポック、学習損失、テスト損失、学習正解率、テスト正解率、経過時間
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.358723    0.33355                                                        0.108125      
2           0.328003    0.319141                                                       0.400148      
3           0.308625    0.309541                                                       0.736793      
4           0.298308    0.309983                                                       1.09959       
5           0.290337    0.308601                                                       1.42175       
6           0.284335    0.308748                                                       1.83863       
7           0.281414    0.306491                                                       2.17969       
8           0.27684     0.30461                                                        2.56712       
9           0.276264    0.308157                                                    